In [1]:
import pandas as pd
import numpy as np

import ast

In [2]:
files = [
    'financial_reports_112_01',
    'financial_reports_112_02',
    'financial_reports_112_03',
    'financial_reports_112_04',
    'financial_reports_112_05',
    'financial_reports_112_06',
    'financial_reports_121'
]

columns_names = {
    'prilohy': 'attachments',
    'id': 'financial_report_id',
    'idUctovnejZavierky': 'financial_statement_id',
    'obsah': 'content',
    'pristupnostDat': 'data_access',
    'idSablony': 'template_id',
    'datumPoslednejUpravy': 'modified',
    'zdrojDat': 'source',
    'kodDanovehoUradu': 'tax_office_code'}

templates_table = pd.read_csv('../../DATA/RAW/register_uz_apr23/preprocessed/templates_dataset.csv')
templates = (templates_table[['template_id', 'template_name', 'table_name']]
 .drop_duplicates()
 .groupby(['template_id', 'template_name'])
 ['table_name'].apply(list)
 .to_frame('tables')
 .reset_index())

In [3]:
df = pd.DataFrame()

for file in files:
    print(file)
    temp = pd.read_csv(f'../../DATA/RAW/register_uz_apr23/preprocessed/{file}.csv')
    if file == 'financial_reports_121':
        temp.drop(columns='idUctovnejZavierky', inplace=True)
    temp.rename(columns=columns_names, inplace=True)

    df = pd.concat([df, temp], axis = 0)
    temp = None

df = df.merge(templates, on = 'template_id', how = 'left')
df

financial_reports_112_01
financial_reports_112_02
financial_reports_112_03
financial_reports_112_04
financial_reports_112_05
financial_reports_112_06
financial_reports_121


,financial_report_id,financial_statement_id,content,data_access,template_id,tax_office_code,source,modified,attachments,template_name,tables
0,4083839,2083688,"{'tabulky': [{'nazov': {'sk': 'Strana aktív'},...",Verejné,21.0,100.0,FRSR,2014-05-16,NaN,Súvaha Úč POD 1-01,"[Strana aktív, Strana pasív]"
1,4083841,2083688,{'tabulky': [{'nazov': {'sk': 'Výkaz ziskov a ...,Verejné,22.0,100.0,FRSR,2014-05-16,NaN,Výkaz ziskov a strát Úč POD 2-01,[Výkaz ziskov a strát]
2,4647638,2384908,"{'tabulky': [{'nazov': {'sk': 'Strana aktív', ...",Verejné,699.0,100.0,FRSR,2015-04-10,NaN,Úč POD,"[Strana aktív, Strana pasív, Výkaz ziskov a st..."
3,4815576,2384908,{'titulnaStrana': {'nazovUctovnejJednotky': 'T...,Verejné,697.0,100.0,FRSR,2015-06-13,NaN,NaN,NaN
4,5107721,2744533,"{'tabulky': [{'nazov': {'sk': 'Strana aktív', ...",Verejné,699.0,100.0,FRSR,2016-03-30,NaN,Úč POD,"[Strana aktív, Strana pasív, Výkaz ziskov a st..."
...,...,...,...,...,...,...,...,...,...,...,...
2573141,8150261,5192577,"{'tabulky': [{'nazov': {'sk': 'Strana aktív'},...",Verejné,687.0,500.0,FRSR,2022-06-21,"[{'id': 9311095, 'meno': 'Príloha k účtovnej z...",Úč MUJ,"[Strana aktív, Strana pasív, Výkaz ziskov a st..."
2573142,8044641,5110158,"{'tabulky': [{'nazov': {'sk': 'Strana aktív'},...",Verejné,687.0,200.0,FRSR,2022-04-03,"[{'id': 9137672, 'meno': 'Príloha k účtovnej z...",Úč MUJ,"[Strana aktív, Strana pasív, Výkaz ziskov a st..."
2573143,7989563,5063068,"{'tabulky': [{'nazov': {'sk': 'Strana aktív'},...",Verejné,687.0,100.0,FRSR,2022-03-29,"[{'id': 9049381, 'meno': 'Príloha k účtovnej z...",Úč MUJ,"[Strana aktív, Strana pasív, Výkaz ziskov a st..."
2573144,4614507,2357676,"{'tabulky': [{'nazov': {'sk': 'Strana aktív', ...",Verejné,699.0,600.0,FRSR,2015-04-01,"[{'id': 4056587, 'meno': 'Príloha k účtovnej z...",Úč POD,"[Strana aktív, Strana pasív, Výkaz ziskov a st..."


In [21]:
# df['template_name'] = df['template_name'].fillna('NEZNAMA')
df.query('template_id in [687, 699, 22, 21, 20, 943] and content == content', inplace=True)
display(df['template_name'].value_counts())

Úč MUJ                              1101354
Úč POD                               661820
Súvaha Úč POD 1-01                   191846
Výkaz ziskov a strát Úč POD 2-01     191828
Výkaz vybraných údajov                  395
Name: template_name, dtype: int64

In [25]:
df.to_csv('../../DATA/TRANSFORM/financial_statements/financial_reports_table.csv', index = False)
df.sample(10000).to_csv('../../DATA/TRANSFORM/financial_statements/financial_reports_table_sample.csv', index = False)